<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">Chapitre 20 : Représentation des nombres flottants</h1>

Nous avons vu que Python permet de calculer avec des nombres décimaux particuliers appelés **nombres flottants**. Ces nombres ont un encodage très compact, sur 32 ou 64 bits, qui permet de représenter des nombres très grand, bien au-delà de ce qu'il est possible de représenter avec un cadage des entiers sur le même nombre de bits, mais également de très petit nombres.

### Ecriture scientifique
L'encodage des nombres flottants est inspiré de l'écriture scientifique des nombres décimaux qui se compose d'un signe (`+` ou `-`), d'un nombre décimal $m$ appelé **mantisse**, compris dans l'intervalle $[1;10[$ et d'un entier relatif $n$ appelé **exposant**.

Par exemple, avec cette notation,

|        |  |        |
|------------:|:-------:|----------------------------|
|       2 156 | s'écrit | $+2,156 \times 10^3$       |
| -398 879,62 | s'écrit | $-3,988 796 2 \times 10^5$ |
|   0,000 142 | s'écrit | $+1,42 \times 10^{-4}$     |
|        1,34 | s'écrit | $+1,34 \times 10^0$        |

Ainsi, de manière générale, l'écriture scientifique d'un nombre décimal est de la forme :
$$\pm m \times 10^n$$
avec $m$ la **mantisse** et $n$ l'**exposant**.  
On note que le nombre 0 ne peut pas être représenté avec cette écriture.

## Norme IEEE 754
La représentation des nombres flottants et les opérations arithmétiques qui les accompagnent ont été définies dans la norme internationale [IEEE](https://interstices.info/glossaire/IEEE/) 754.  
C'est la norme la plus couramment utilisée dans les ordinateurs.  
Selon la précision ou l'intervalle de représentation souhaité, la norme définit un format de données sur 32 bits, appelé **simple précision** ou **binary32**, et un autre sur 64 bits, appelé [**double précision**](https://interstices.info/glossaire/double-precision/) ou **binary64**.  
Dans les deux cas, la représentation d'un nombre flottant est similaire à l'écriture scientifique d'un nombre décimal, à savoir une décomposition en trois partie :
* un signe $s$
* une mantisse $m$
* un exposant $n$
De manière générale, un nombre flottant a la forme suivante.
$$(-1)^s\times m \times 2^{(n-d)}$$

Les différences entre la norme IEEE 754 et l'écriture scientifique sont : 
* la base choisie est la base 2
* la mantisse est dans l'intervalle $[1;2[$
* l'exposant $n$ est *décalé* (ou biaisé) d'une valeur $d$ qui dépend du format choisi (32 ou 64 bits)

Dans le format 32 bits :
* le bit de poids fort est utilisé pour représenter le signe $s$ (avec 0 pour le signe +)
* les 8 bits suivants sont reservés pour stocker la valeur de l'exposant $n$
* les 23 derniers bits servent à décrire la mantisse

            1              8                        23
        +-------+--------------------+------------------------------+
        | signe |      exposant      |           mantisse           |
        +-------+--------------------+------------------------------+

Afin de représenter des exposants négatifs et positifs, la norme IEEE 754 n'utilise pas l'encodage par complément à 2 des entiers relatifs, mais une technique qui consiste à stocker l'exposant de manière décalé sous la forme d'un nombre non signé.  
Ainsi l'exposant décalé $n$ est un entier sur sur 8 bits qui représente des entiers entre 0 et 255.  
Pour le format 32 bits, l'exposant est décalé avec $d=127$, ce qui permet de représenter les entiers de l'intervalle $[-127, 128]$. Néanmoins, les valeurs 0 et 255 étant reservées pour représenter des nombres particuliers, les exposants sont donc ceux de l'intervalle $[-126, 127]$.

La mantisse $m$ étant toujours comprise dans l'intervalle $[1;2[$, elle représente un nombre de la forme $1,xx...xx$, c'est-à-dire commençant nécessairement par le chiffre 1.  
Par conséquent, pour gagner 1 bit de précision, les 23 bits dédiés à la mantisse sont uniquement utilisés pour représenter les chiffres **après** la virgule, qu'on appelle la **fraction** .   
Ainsi si les 23 bits dédiés à la mantisse sont $b_1b_2...b_{23}$, alors la mantisse représente le nombre :
$$1+b_1 \times 2^{-1}+b_2 \times 2^{-2} + ...+b_{23}\times 2^{-23}$$

Par exemple, le mot de 32 bits suivant 

| signe | exposant |         mantisse        |
|:-----:|:--------:|:-----------------------:|
|   1   | 10000110 | 10101101100000000000000 |

représente le nombre décimal calculé ainsi :
* signe : $(-1)^1 = -1$
* exposant : $(2^7+2^2+2^)-127 = (128+4+2)-127=134-127=7$
* mantisse : $1+2^{-1}+2^{-3}+2^{-5}+2^{-6}+2^{-8}+2^{-9}=1,677 734 375$

Soit, au final, le nombre :
$$-1,677 734 375 \times 2^7=-214,75$$

Concernant les deux formats, la différence entre les encodages 32 et 64 bits est simplement la valeur $d$ du décalage pour l'exposant et le nombre de bits alloués pour la fraction $f$ de la mantisse $m$ et l'exposant $n$.

|         | exposant : $e$ | fraction : $f$ |                valeur                |
|:-------:|:--------------:|:--------------:|:------------------------------------:|
| 32 bits |     8 bits     |     23 bits    |  $$(-1)^s\times 1,f \times 2^{(e-127)}$$ |
| 64 bits |     11 bits    |     52 bits    | $$(-1)^s\times 1,f \times 2^{(e-1023)}$$ |

Ainsi, en simple précision (32 bits), les nombes flottants positifs peuvent représenter les nombres décimaux compris (approximativement) dans l'intervalle $[10^{-38}; 10^{38}]$, tandis qu'en double précision, l'intervalle des nombres décimaux représentable est (approximativement) $[10^{-308}; 10^{308}]$.

### Valeurs spéciales
Tel que nous l'avons défini jusqu'ici, le format des nombres flottants ne permet pas de représenter le nombre 0. En effet, puisqu'un nombre flottant sur 32 bits correspond à la formule $(-1)^s\times 1,f \times 2^{(e-127)}$, la forme $1,f$ de la mantisse interdit la représentation du 0.  
Pour remédier à ce problème, la norme IEEE 754 utilise les valeurs de l'exposant restées jusqu'à présent inutilisées, à savoir 0 et 255, pour représenter le nombre 0, mais aussi d'autres valeurs spéciales :
* Le nombre 0 est représenté par l'exposant 0 et la mantisse 0. La norme reconnaît le 0 positif et le 0 négatif.
* La norme permet de représenter deux infinis, en utilisant l'exposant 255 et la mantisse 0. Le bit de signe est utilisé pour représenter le signe de ces infinis.  
Ces infinis sont utilisés pour indiquer des dépassements de capacité.
* Une valeur spéciale, $NaN$ (Not a Number), permet de représenter les résultats d'opérations invalides comme $\dfrac{0}{0}$, $\sqrt{-1}$, $0 \times \infty$, ... Cette valeur est encodée avec le signe 0, l'exposant 255 et n'importe quel mantisse différente de 0.

| signe | exposant | fraction | valeur spéciale |
|:-----:|:--------:|:--------:|:---------------:|
|   0   |     0    |     0    |       $+0$      |
|   1   |     0    |     0    |       $-0$      |
|   0   |    255   |     0    |    $+\infty$    |
|   1   |    255   |     0    |    $-\infty$    |
|   0   |    255   | $\neq 0$ |      `NaN`      |



### Nombre dénormalisés
Si l'exposant d'un nombre flottant (sur 32 bits) est compris entre 1 et 254, alors la valeur représentée par l'encodage est $(-1)^s\times 1,f \times 2^{(e-127)}$. Les nombres représentés ainsi sont les nombres flottants **normalisés**.   
Avec cet encodage, le plus petit nombre décimal positif représentable est dont $2^{-126}$ (soit environ $10^{-38}$).  
Puisque la mantisse est implicitement de la forme $1,f$, il n'y a pas de nombres représentables dans l'intervalle $[0, 2^{-126}[$, alors qu'il y en a $2^{23}$ dans l'intervalle $[1 \times 2^{-126}; 2 \times 2^{-126}]=[2^{-126}; 2^{-125}]$.

Afin de rééquilibrer la représentation des nombres autour de 0, la norme IEEE 754 permet d'encoder des nombres de la forme :
$$(-1)^s\times 0,f \times 2^{-126}$$
avec une mantisse $0,f$ commençant implicitement par 0.  
Ces nombres flottants, appelés nombres **dénormalisés**, correspondent à des nombres flottants avec un exposant à 0 et une mantisse différente de 0.  
De cette manière, la plus petite valeur représentable avec des nombres dénormalisés est $2^{-23} \times 2^{-126} = 2^{-149}$.

#### Test d'égalité
Sans les nombres dénormalisés, les deux tests $x-y=0$ et $x=y$ ne seraient pas systématiquement équivalent.

### Arrondis
Il est important de noter que la représentation des nombre décimaux par des flottants est une représentation **approximative**.   
Par exemple, le nombre décimal $1,6$ ne peut pas être représenté exactement avec cet encodage. Il faut donc **arrondir** cette valeur en choisissant le meilleur nombre flottant pour la représenter.

La norme IEEE 754 définit quatre modes d'[arrondi](https://interstices.info/glossaire/arrondi/) pour choisir le meilleur flottant :
* *au plus près* : le flottant le plus proche de la valeur exacte (en cas d'égalité, on privilégie le flottant pair, c'est-à-dire avec une mantisse se terminant par 0
* *vers zéro* : le flottant le plus proche de zéro
* *vers plus l'infini* : le plus petit flottant supérieur ou égal à la valeur exacte
* *vers moins l'infini* : le plus grand flottant inférieur ou égal à la valeur exacte

Le mode d'arrondi, par défaut, de la norme est *au plus prêt*.  
Par exemple, le nombre flottant le plus proche de $1,6$ est :
$$0~01111111~10011001100110011001101$$
qui correspond au nombre décimal :
$$(2^{-1}+2^{-4}+2^{-5}+2^{-8}+2^{-9}+2^{-12}+2^{-13}+2^{-16}+2^{-17}+2^{-20}+2^{-21}+2^{-23})\times 2^{(127-127)}$$  
soit $1,60000002384185791015625$

Cette opération d'arrondi se propage également aux opérations arithmétiques. En effet, même si deux nombres décimaux sont exactement représentables avec des flottants, il n'en est pas de même pour le résultat d'une opération entre ces deux nombres.  
Ainsi le norme IEEE 754 exige, pour les opérations usuelles (addition, multiplication, soustraction, division, racine carrée), la propriété d'**arrondi correct**, à savoir que le résultat obtenu en appliquant un opérateur sur deux nombres flottants est le même que celui qu'on obtiendrait en effectuant l'opération en précision infini sur ces deux nombres puis en arrondissant.

## Les flottants en Python
Les [flottants](https://docs.python.org/fr/3/tutorial/floatingpoint.html) sont, généralement, représentés selon la norme IEEE 754 double précision (format 64 bits).

In [ ]:
x = 1.6
1.2e-4

La fonction [`float`](https://docs.python.org/fr/3/library/functions.html?#float) permet de convertir un entier en un flottant.  
Inversement, la fonction [`int`](https://docs.python.org/fr/3/library/functions.html?#int) transforme un flottant en un entier, en ignorant la partie après la virgule.

In [ ]:
float(-4)

In [ ]:
int(5.9)

Les opérations arithmétiques usuelles s'appliquent également aux flottants

In [ ]:
3.4 + 0.012

In [ ]:
1.2 / 2.0

L'opérateur de division `/` produit toujours un résultat flottant, quelque soit le type de ses arguments.

In [ ]:
1 / 2

In [ ]:
4 / 2

In [ ]:
2 / 3.2

Certaines expressions peuvent générer des valeurs spéciales, comme `inf` ou `nan`, pour représenter respectivement des **dépassements de capacité** ou des **opérations non valides**.

In [ ]:
x = 1e200
x * x

In [ ]:
(x * x) * 0

Des conversions d'entiers vers flottants peuvent être réalisées de manière implicite dans les expressions arithmétiques où un opérateur s'applique à la fois à un entier et un flottant

In [ ]:
5 * 2.7

Ces conversions peuvent procoquer des erreurs qu'il est important de bien différencier des débordements de capacité sur les flottants.  
L'exécution du code

```python
>>> y = 10 ** 400
>>> y + 0.5
```
provoque une erreur

```python
OverflowError: int too large to convert to float
```

La variable `y` contient un entier après son initialisation à `10 ** 400`. Cet entier doit être converti (de manière implicite) en un flottant afin que l'addition `y + 0.5` puisse être exécutée. Cette conversion provoque une erreur.

La bibliothèque standard de Python fournit également de nombreuses fonctions mathématiques rassemblées dans le module [`math`](https://docs.python.org/fr/3/library/math.html?#module-math)

In [ ]:
import math

In [ ]:
math.sqrt(9)

In [ ]:
math.sin(1.5 * math.pi)

On peut afficher des nombres flottants en précisant le nombre de chiffres après la virgule à l'aide d'une [chaîne de formatage](https://docs.python.org/fr/3/reference/lexical_analysis.html#f-strings) en utilisant la spécification `[precision].f` où `[precision]` est une constante entière (positive).

In [ ]:
x = 0.12345
print("%.2f" % x) # anciennement uitilisé

In [ ]:
x = 0.12345
f"x = {x :.2f}" # en utilisant les f-string

In [ ]:
f"{1.6 :.20f}"

### Propriété des nombres flottants
Il faut être très prudent lorsque l'on manipule des nombres flottants.  En effet, certaines opérations sur ces nombres n'ont pas les mêmes propriétés que sur les nombres réels et il ne faut jamais oublier que les calculs sont inexacts.

In [ ]:
1.2 * 3

En ce qui concerne les propriétés des oparations, l'addition et la multiplication ne sont pas associatives.

In [ ]:
x = 1.6 + (3.2 + 1.7)
f"x = {x : .20f}"

In [ ]:
x = (1.6 + 3.2) + 1.7
f"x = {x : .20f}"

La multiplication n'est pas distributive par rapport à l'addition.

In [ ]:
x = 1.5 * (3.2 + 1.4)
f"x = {x : .20f}"

In [ ]:
x = 1.5 * 3.2 + 1.5 * 1.4
f"x = {x : .20f}"

Il est donc très imprudent  d'écrire des programmes dans lesquels on utilise des tests d'égalité entre flottants.

In [ ]:
0.1 + 0.2 == 0.3

Aussi, plutôt que d'écrire un test d'égalité entre deux valeurs flottantes $v_1$ et $v_2$, il est préférable d'écrire un test d'égalité $|v_1-v_2|<\epsilon$ antre la valeur absolue de la différence $v_1-v_2$ et une borne de précision $\epsilon$.  
Par exemple, pour tester l'égalité à $10^{-12}$ prés entre les variables `x` et `y`, o, écrira

In [ ]:
x = 0.1 + 0.2
y = 0.3
abs(x - y) < 1e-12

#### Information sur les flottants
Le module [`sys`](https://docs.python.org/fr/3/library/sys.html?#module-sys) de la bibliothèque standard permet d'accéder à un $n$-uplet [`float_info`](https://docs.python.org/fr/3/library/sys.html#sys.float_info) contenant des informations à propos des nombres flottants de Python.

In [ ]:
from sys import float_info
float_info

#### Alternatives pour les calculs
Aussi, comme on le voit, les différentes erreurs d’arrondi qui se produisent à chaque étape du calcul s’accu-
mulent et produisent un résultat parfois surprenant. Ce problème n’est pas spécifique à Python, il existe pour tous les langages, et il est bien connu des numériciens.  
Dans une grande majorité des cas, ces erreurs d’arrondi ne sont pas pénalisantes. Il faut toutefois en être
conscient car cela peut expliquer des comportements curieux.

* Tout d’abord si votre problème se pose bien en termes de nombres rationnels, il est alors tout à fait possible de le résoudre avec exactitude.  
Alors qu’il n’est pas possible d’écrire exactement 3/10 en base 2, ni d’ailleurs 1/3 en base 10, on peut représenter exactement ces nombres dès lors qu’on les considère comme des fractions et qu’on les encode
avec deux nombres entiers.  
Python fournit en standard le module [`fractions`](https://docs.python.org/fr/3/library/fractions.html?#module-fractions) qui permet de résoudre le problème.

In [ ]:
from fractions import Fraction

Fraction(3, 10) - Fraction(1, 10) == Fraction(2, 10)

* Si par contre vous ne manipulez pas des nombres rationnels et que du coup la représentation sous forme
de fractions ne peut pas convenir dans votre cas, le module standard [`decimal`](https://docs.python.org/fr/3/library/decimal.html?#module-decimal) offre des fonctionnalités très voisines du type `float`, tout en éliminant la plupart des inconvénients, au prix naturellement d’une consommation mémoire supérieure.

In [ ]:
from decimal import Decimal

Decimal('0.3') - Decimal('0.1') == Decimal('0.2')

## Exercices

### Exercice 1
Donner la représentation flottante, en précision simple, de $128$ et $-32,75$.

### Exercice 2
Donner la valeur décimale des nombres flottants suivants codés en simple précision :
* 1 01111110 11110000000000000000000
* 0 10000011 11100000000000000000000

### Exercice 3
On tape en Python l'expression arithmétique suivante :

```python
>>> (1e25 + 16) - 1e25
```
Quel est le résultat attendu?  
Quel est le résultat obtenu?  
Pourquoi?

### Exercice 4
On tape, en Python, les instructions suivantes :

```python
>>> x = 1e2000
>>> y = x * x
>>> z = y / y
```
Quelles sont les valeurs de `y` et `z` ?   
Pourquoi?

### Exercice 5
Ecrire un programme qui, partant du nombre `1.0`, le divise vingt fois de suite par `3`, puis le multiplie vingt fois de suite par `3`, puis enfin affiche la valeur obtenue au final.  
Expliquer le résultat observé.

### Exercice 6 : [méthode de Monte Carlo](https://www.geogebra.org/m/KMPYb3pM)
Nous allons écrire un programme qui estime la valeur de $\pi$ de la façon suivante.
* On répète un grand nombre de fois l'opération consistant à choisir au hasard un point dans un carré de côté 1. 
* On compte combien de points tombent dans le quart de cercle de rayon 1 centré sur l'un des coins du carré.
* Le rapport de ce nombre au nombre total de point donne une estimation de $\dfrac{\pi}{4}$.  

### Exercice 7
1. Trouver un nombre (flottant) `x` tel que `x + 1 == x`.
2. Trouver le plus petit de ces nombres.

### Exercice 8
1. Trouver un nombre flottant `x` différent de `0` tel que `2.0 ** 100 + x == 2.0 ** 100`.
2. Trouver le plus grand de ces nombres.

## Sources :
* Balabonski Thibaut, et al. 2019. *Spécialité Numérique et sciences informatiques : 30 leçons avec exercices corrigés - Première - Nouveaux programmes*. Paris. Ellipse
* [IEEE Standard for Floating-Point Arithmetic](Ressources/Standards/IEEE%20Standard%20for%20Floating-PointArithmetic.pdf)
* Documentation Python : [Types numériques](https://docs.python.org/fr/3/library/stdtypes.html#numeric-types-int-float-complex)
* Documentation Python : [Arithmétique en nombres à virgule flottante](https://docs.python.org/fr/3/tutorial/floatingpoint.html)
* Documentation Python : FAQ [Pourquoi les calculs à virgules flottantes sont si imprécis ?](https://docs.python.org/fr/3/faq/design.html#why-are-floating-point-calculations-so-inaccurate)
* Image des mathématiques : [Erreurs en arithmetique des ordinateurs](http://images.math.cnrs.fr/Erreurs-en-arithmetique-des.html)
* Interstices : Podcast : [Pourquoi mon ordinateur calcule-t-il faux?](https://interstices.info/pourquoi-mon-ordinateur-calcule-t-il-faux/)
* Interstices : [Tout a un reflet numérique](https://interstices.info/tout-a-un-reflet-numerique/)
* [What every computer scientist should know about floating-point arithmetic](Ressources/Lectures/What%20every%20computer%20scientist%20should%20know%20about%20floating-point%20arithmetic%20(Goldberg)%201991.pdf), Goldberg (1991)
* [The Perils of Floating Point](http://www.lahey.com/float.htm) 
* Wolfram : [Conversion](https://www.wolframalpha.com/input/?i=radix&assumption=%7B%22C%22%2C+%22radix%22%7D+-%3E+%7B%22Calculator%22%7D&assumption=%7B%22F%22%2C+%22BaseConversion%22%2C+%22fromBase%22%7D+-%3E%2210%22&assumption=%7B%22F%22%2C+%22BaseConversion%22%2C+%22toBase%22%7D+-%3E%222%22&assumption=%7B%22F%22%2C+%22BaseConversion%22%2C+%22numToConvert%22%7D+-%3E%221.52%22)
* Floating point precision : [bugs](https://www.nsc.liu.se/wg25/book/ch1/#1)
* Convertisseur [en ligne](http://www.binaryconvert.com/)
* Editeur hexadécimal [en ligne](https://hexed.it/)